In [99]:
import os
import csv
from os import path
import pandas as pd
import locale
import numpy as np
import math

#pd.set_option('display.max_rows', 500)
pd.reset_option('display.max_rows')

In [53]:
def get_full_file_path(file_path_list, file_name):
    """
    get_full_file_path returns os-specific path for a given filename and path
    :param file_path_list: file path
    :param file_name: file name
    :return: file path string
    """
    path_list = file_path_list + [file_name]
    return os.path.join(*path_list)  # splat * to unpack list arg

def import_csv(filename, import_path=['./']):
    """
    ingest_csv tries to import and read a csv file
    :param filename: string containing filename
    :param import_path: string for specified input folder
    :return: list of data
    """
    full_filepath = get_full_file_path(import_path, filename)  # assume files are in specific directory
    try:
        if path.isfile(full_filepath):
            # copied from https://automatetheboringstuff.com/chapter14/
            file = open(full_filepath, encoding="utf-8")
            reader = csv.reader(file)
            data = list(reader)
            file.close()
            return data
        else:
            raise Exception(FileNotFoundError, full_filepath + ' is not found')
    except Exception as instance:
        return str(instance)
    
def get_dataframe_from_list(data, headers=None):
    """
    get_dataframe_from_list translates raw data to pandas dataframe
    :param data: raw data to convert
    :param headers: list of strings for column titles
    :return: dataframe
    """
    if headers is None:
        headers = data.pop(0)
    return pd.DataFrame(data, columns=headers)

In [54]:
column_id = "id"
column_name = 'name'
column_top_ten = 'top_ten'
column_top_theme = 'top_theme'
column_bottom_five = 'bottom_five'
column_bottom_theme = 'bottom_theme'

In [92]:
# import and clean the data
raw_data = import_csv('data.csv')
raw_headers = raw_data.pop(0)
df = get_dataframe_from_list(raw_data, raw_headers)
headers = list(filter(lambda x: len(x) > 0, raw_headers)) # remove empty strings from headers
df.drop(df.columns[range(len(headers), len(raw_headers)-1)], axis =1, inplace=True) # crop dataframe to only desired data
df = df.set_axis([column_id,column_name,column_top_ten,column_top_theme, column_bottom_five, column_bottom_theme], axis='columns')
df

,id,name,top_ten,top_theme,bottom_five,bottom_theme
0,1,28,Achiever,Executing,Context,Strategic Thinking
1,2,28,Competition,Influencing,Restorative,Executing
2,3,28,Strategic,Strategic Thinking,Deliberate,Executing
3,4,28,Relator,Relationship Building,Empathy,Relationship Building
4,5,28,Learner,Strategic Thinking,Connectedness,Relationship Building
...,...,...,...,...,...,...
215,216,21,Intellection,Strategic Thinking,,-
216,217,21,Connectedness,Relationship Building,,-
217,218,21,Analytical,Strategic Thinking,,-
218,219,21,Relator,Relationship Building,,-


In [102]:
# create separate dataframe for bottom results
bottom_df = df[[column_id, column_name,column_bottom_five, column_bottom_theme]].copy()
bottom_df.drop(bottom_df[bottom_df[column_bottom_five].str.len().lt(2)].index, inplace=True) # remove empty rows
bottom_df

,id,name,bottom_five,bottom_theme
0,1,28,Context,Strategic Thinking
1,2,28,Restorative,Executing
2,3,28,Deliberate,Executing
3,4,28,Empathy,Relationship Building
4,5,28,Connectedness,Relationship Building
...,...,...,...,...
210,211,21,Restorative,Executing
211,212,21,Competition,Influencing
212,213,21,Deliberate,Executing
213,214,21,Consistency,Executing


In [101]:
# create separate dataframe for top results
top_df = df[[column_id, column_name,column_top_ten, column_top_theme]].copy()
top_df.drop(top_df[top_df[column_top_ten].str.len().lt(2)].index, inplace=True) # remove empty rows
top_df

,id,name,top_ten,top_theme
0,1,28,Achiever,Executing
1,2,28,Competition,Influencing
2,3,28,Strategic,Strategic Thinking
3,4,28,Relator,Relationship Building
4,5,28,Learner,Strategic Thinking
...,...,...,...,...
215,216,21,Intellection,Strategic Thinking
216,217,21,Connectedness,Relationship Building
217,218,21,Analytical,Strategic Thinking
218,219,21,Relator,Relationship Building


In [ ]:
# 1) Would be great to figure out how we can visualize the worksheet called “Top 10_Bottom 5”

In [127]:
# 2) Which of the 34 strengths appear in people’s list of Top 10 strengths most frequently? 
top_strengths = top_df[column_top_ten].value_counts()
top_strengths

Relator              16
Learner              14
Individualization    14
Arranger             13
Strategic            13
Futuristic           11
Maximizer            10
Input                10
Ideation              9
Connectedness         9
Activator             9
Achiever              8
Developer             7
Intellection          6
Responsibility        6
Positivity            6
Focus                 4
Communication         4
Adaptability          4
Analytical            4
Self-Assurance        3
Empathy               3
Competition           3
Discipline            3
Restorative           3
Context               3
Significance          3
Command               3
Belief                2
Woo                   2
Harmony               2
Includer              2
Consistency           1
Name: top_ten, dtype: int64

In [106]:
# 2a) What percentage of people have each of these ‘most common top-10 strengths’?

In [124]:
# 3) Which of the 34 strengths appear in people’s list of Bottom 5 strengths most frequently?
bottom_df[column_bottom_five].value_counts()


Harmony              11
Deliberate           10
Consistency          10
Context               9
Restorative           8
Competition           6
Discipline            6
Significance          5
Empathy               5
Adaptability          5
Includer              4
Woo                   4
Input                 3
Maximizer             3
Strategic             2
Connectedness         2
Individualization     1
Analytical            1
Achiever              1
Ideation              1
Intellection          1
Communication         1
Command               1
Name: bottom_five, dtype: int64

In [108]:
# 3a) What percentage of people have each of these ‘most common bottom-5 strengths’?


In [122]:
# 4) Which of the four Themes is the most common Theme represented in the Top-10 Strengths? Second most? Third? Fourth?
top_df[column_top_theme].value_counts()


Strategic Thinking       70
Relationship Building    63
Executing                40
Influencing              37
Name: top_theme, dtype: int64

In [110]:
# 4a) How many people have each theme represented?


In [123]:
# 5) Which of the four Themes is the most common Theme represented in the Bottom-5 Strengths? Second most? Third? Fourth?
bottom_df[column_bottom_theme].value_counts()


Executing                35
Relationship Building    28
Influencing              20
Strategic Thinking       17
Name: bottom_theme, dtype: int64

In [112]:
# 5a) How many people have each theme represented?


In [129]:
# 6) Is there a Strength that no one has or very few people have in their Top-10?
unique_top_strengths = top_strengths[top_strengths == 1]
unique_top_strengths

Consistency    1
Name: top_ten, dtype: int64

In [114]:
# 7) Is there a Strength that no one has or very few people have in their Bottom-5?


In [116]:
# 8) Is there a Strength that only one person has in their Top-10, and that same strength is in other people’s Bottom-5?
# I think Consistency is this one


In [117]:
# 9) Who are the people who have Strengths in their Top-10 that are represented among all four Themes?


In [118]:
# 9a) Are there people whose Top-10 Strengths are only among three Themes? Can we see which Themes?


In [119]:
# 9b) What about people who only have strengths in two themes? Or just in one theme?
